In [ ]:
!pip install pandas
!pip install torch
!pip install transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final.csv to final.csv


In [ ]:
# Load the dataset
df = pd.read_csv('final.csv')

In [ ]:
# Combine the input columns into a single text with special tokens
df['text'] = '[COMPANY]' + df['company_name'] + ' [MESSAGE]' + df['company_message'] + ' [MODEL]' + df['car_model'] + ' [DESCRIPTION]' + df['car_description'] + ' [PRICE]' + df['car_price'].astype(str) + ' [MODE OF CAMPAIGN]' + df['type_mode_of_campaign']

In [ ]:
df['target'] = '[DESCRIPTION]' + df['description_of_campaign'] + ' [CONCEPT]' + df['concept_of_campaign'] + ' [SLOGAN]' + df['slogan_of_campaign'] + ' [TARGET AUDIENCE]' + df['target_audience_of_campaign'] + ' [REASON]' + df['reason']

In [ ]:
# Create a new column that combines the input and output texts
df['data'] = df['text'] + ' ' + df['target']

In [ ]:
# Save the combined data to a text file
df['data'].to_csv('train.txt', header=False, index=False, sep='\t')

In [ ]:
# Initialize the GPT2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Prepare the training dataset and the data collator
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Initialize the Trainer with its training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-car-ad", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output directory
    num_train_epochs=3, # The number of epochs
    per_device_train_batch_size=1, # The batch size for training
    save_steps=10_000, # The number of update steps before checkpoint saves
    save_total_limit=2, # The maximum amount of checkpoints
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,2.634300
1000,2.306000
1500,2.181600
2000,2.097700
2500,2.039500
3000,2.033500
3500,1.930800
4000,1.892200
4500,1.714600
5000,1.720300


TrainOutput(global_step=12537, training_loss=1.7362042101716808, metrics={'train_runtime': 1100.8571, 'train_samples_per_second': 11.388, 'train_steps_per_second': 11.388, 'total_flos': 818954551296000.0, 'train_loss': 1.7362042101716808, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./gpt2-car-ad")

In [ ]:
tokenizer.save_pretrained("./gpt2-car-ad")

('./gpt2-car-ad/tokenizer_config.json',
 './gpt2-car-ad/special_tokens_map.json',
 './gpt2-car-ad/vocab.json',
 './gpt2-car-ad/merges.txt',
 './gpt2-car-ad/added_tokens.json')

In [ ]:
# Load the trained model and the tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2-car-ad")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-car-ad")

In [ ]:
from google.colab import files
files.download("/content/gpt2-car-ad.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
prompt = "[COMPANY] Toyota [MESSAGE] Our mission is to provide the best cars [MODEL] Innova Crysta [DESCRIPTION] It's a family car with high reliability [PRICE] Starting at 200,000 [CAMPAIGN] Online on social media"

In [ ]:
# Encode the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [ ]:
# Generate text
output = model.generate(input_ids, max_length=1000, temperature=0.7, num_return_sequences=1, repetition_penalty=1.2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print(generated_text)

[COMPANY] Toyota [MESSAGE] Our mission is to provide the best cars [MODEL] Innova Crysta [DESCRIPTION] It's a family car with high reliability [PRICE] Starting at 200,000 [CAMPAIGN] Online on social media and search engines.
* **Search Engine Optimization (SEO):** Running paid ads in search engine results pages to improve visibility and drive traffic to our website for you! * Search Engines: Targeted advertising campaigns across various websites and platforms to reach your desired audience. 
* Social Media Marketing Campaigns: We use Facebook Ads as part of this marketing campaign that helps us connect with our target audience and generate leads.
* Influencer Partnerships: Collaborating directly or through trusted influencers to create authentic reviews and build credibility.

**Our Values and Commitment**

We believe driving a sustainable and fulfilling life can be both empowering AND rewarding. The Toyota Innovas embodies these values:

* Value individuality; aspiration towards a veh